In [ ]:
# ===========================
# Cell 0 — Config (Stacked QAE, 6q_4l_2t, tau30)
# ===========================
import numpy as np

DATA_ID = 'mackey_glass_tau17_n200'

# Architecture (4q = 3 latent + 1 trash)
wind_size = 4
wind_step = 2
n_latent = 3
n_trash = wind_size - n_latent  # = 1

# Ansatz
rot = 'Rxyz'

# Training instances
LAYER_OPTIONS = [1,3,4]
INSTANCES_PER_LAYER = [1,2,3,4,5,6,7]

# Data parameters
noise = 0.2
split = 0.75
data_low = 0
data_high = 1

# Encoding/decoding ranges (no margin, clip noise instead)
y_margin = 0.0
y_enc_low, y_enc_high = -np.pi + y_margin, 0 - y_margin
y_dec_low, y_dec_high = -1, +1
noise_clip = True

# Simulator settings
sim = 'lightning.qubit'
interface = 'autograd'
diff_method = 'adjoint'
shots = None

# Training parameters - Stage 1 (Inverted Decoder)
n_inv_decoder_epochs = 500
lr_inv_decoder_initial = 0.02
inv_decoder_weight_scaler = 0.05

# Training parameters - Stage 2 (Stacked)
n_stacked_epochs = 300
lr_stacked_initial = 0.1
lr_stacked_decay_rate = 0.75
lr_stacked_decay_steps = 50
stacked_weight_scaler = 0.1
batch_size = 10

# Seeds
TRAIN_SEED_BASE = 88000
TEST_SEED_FIXED = 99123

# Paths (relative to Stacked_QAE folder)
BUNDLE_ROOT = '../../qae_runs'

# Derived parameters
n_wires_full = n_latent + 2*n_trash
log_interv = 1

In [ ]:
# ===========================
# Cell 1 — Imports & Utilities
# ===========================
import sys, os, time, random, json, csv
from pathlib import Path
import matplotlib.pyplot as plt
from pprint import pprint

# Add paths to find qae_utils
sys.path.append('../../..')
sys.path.append('../../../..')

# QAE utilities
from qae_utils.Files import create_folder_if_needed, read_json_file, read_ts_file
from qae_utils.Window import ts_wind_make, ts_wind_split, ts_wind_flatten_avg, ts_add_noise
from qae_utils.Charts import multi_plot_flat_ts, meas_plot
from qae_utils.Tools import draw_circuit

import pennylane as qml
from pennylane import numpy as pnp
import torch

# Create necessary folders
create_folder_if_needed(BUNDLE_ROOT)

np.set_printoptions(suppress=True, precision=6)
plt.rcParams["figure.figsize"] = (8, 4)

%matplotlib inline

# Cell 1 — Update set_global_seed function

def set_global_seed(instance_id: int, layer: int = None):
    """Set reproducible seeds for training instance"""
    if layer is not None:
        base = 10_000 + int(layer) * 1000 + int(instance_id)
    else:
        base = 10_000 + int(instance_id)
    
    random.seed(base + 11)
    np.random.seed(base + 22)
    try:
        pnp.random.seed(base + 33)
    except Exception:
        pass
    
    device_seed = base + 44
    
    return {
        'global_seed': base,
        'numpy_seed': base + 22,
        'pnp_seed': base + 33,
        'device_seed': device_seed
    }

def arch_tag():
    """Generate architecture tag string"""
    return f"{wind_size}q_{n_latent}l_{n_trash}t_stacked"

def x2y(x, xlim=(0, 1), ylim=(0, np.pi)):
    """Scale values from xlim range to ylim range"""
    low_x, high_x = xlim
    low_y, high_y = ylim
    input_range_length = high_x - low_x
    if np.isclose(input_range_length, 0.0):
        return (low_y + high_y) / 2
    scaling_factor = (high_y - low_y) / input_range_length
    return low_y + (x - low_x) * scaling_factor

In [ ]:
# ===========================
# Cell 2 — Load Data
# ===========================
def _find_data_folder(data_id):
    """Find data folder with robust path searching"""
    candidates = [
        f'../../../data/{data_id}',
        f'../../data/{data_id}',
        f'../data/{data_id}',
    ]
    
    tried = []
    for folder in candidates:
        abs_path = os.path.abspath(folder)
        tried.append(abs_path)
        info_p = os.path.join(folder, 'info.json')
        x_p = os.path.join(folder, 'x_org.arr')
        y_p = os.path.join(folder, 'y_org.arr')
        
        if os.path.exists(info_p) and os.path.exists(x_p) and os.path.exists(y_p):
            return folder
    
    raise FileNotFoundError(
        f"Could not find dataset: {data_id}\n"
        f"Tried paths:\n" + "\n".join(f"  - {p}" for p in tried)
    )

DATA_FOLDER = _find_data_folder(DATA_ID)
info = read_json_file(f'{DATA_FOLDER}/info.json')
X = read_ts_file(f'{DATA_FOLDER}/x_org.arr').astype(int)
y = read_ts_file(f'{DATA_FOLDER}/y_org.arr')

scale_low = float(info['scale_low'])
scale_high = float(info['scale_high'])

print(f'Data folder: {os.path.abspath(DATA_FOLDER)}')
print(f'Loaded {len(y)} samples; scale=[{scale_low:.3f}, {scale_high:.3f}]')

# Plot original data
multi_plot_flat_ts(
    X_list=[X], 
    y_list=[y], 
    colors=['blue'], 
    labels=['Original'],
    lines=['solid'], 
    legend_cols=1, 
    rcParams=(10, 3),
    xlabel=f'Range (samples={len(y)})',
    title=f'Stacked QAE Training Data ({DATA_ID})')

In [ ]:
# ===========================
# Cell 3 — Stacked QAE Architecture Components
# ===========================

def sequence_encoder(wires, inputs):
    """Encodes sequence (similar to AngleEncoding)"""
    for i in range(len(wires)):
        if i < len(inputs):
            qml.RY(inputs[i], wires=wires[i])
        else:
            qml.RY(0, wires=wires[i])

def entangler_shape(n_layers, n_wires, rot='Rxyz'):
    """Get shape for entangling layer"""
    if rot == 'Ry':
        return qml.BasicEntanglerLayers.shape(n_layers=n_layers, n_wires=n_wires)
    elif rot == 'Rxyz':
        return qml.StronglyEntanglingLayers.shape(n_layers=n_layers, n_wires=n_wires)

def entangler(weights, wires, rot='Rxyz'):
    """Apply entangling layers"""
    if rot == 'Ry':
        qml.BasicEntanglerLayers(weights, wires=wires, rotation=qml.RY)
    elif rot == 'Rxyz':
        qml.StronglyEntanglingLayers(weights, wires=wires)

def swap(from_wires, to_wires):
    """Apply SWAP gates"""
    for i in range(len(from_wires)):
        qml.SWAP(wires=[from_wires[i], to_wires[i]])

def swap_test(from_wires, to_wires, ancila_wire):
    """Apply SWAP test"""
    qml.Hadamard(wires=ancila_wire)
    for i in range(len(from_wires)):
        qml.CSWAP(wires=[ancila_wire, from_wires[i], to_wires[i]])
    qml.Hadamard(wires=ancila_wire)

# Stage 1: Inverted Decoder Model (same as Sidekick)
def qae_inv_decoder_model_shape(n_latent, n_trash, n_layers=1, rot='Rxyz'):
    return entangler_shape(n_layers, n_latent+n_trash, rot=rot)

def qae_inv_decoder_model(wires, n_latent, n_trash, n_layers=1, rot='Rxyz'):
    n_zero = n_trash
    latent_wires = wires[0:n_latent]
    trash_wires = wires[n_latent:n_latent+n_trash]
    zero_wires = wires[n_latent+n_trash:n_latent+n_trash+n_zero]
    data_wires = latent_wires + trash_wires
    anz_wires = latent_wires + trash_wires
    ancila_wire = wires[-1]

    def _qae_inv_decoder(weights, inputs):
        sequence_encoder(data_wires, inputs)
        qml.Barrier(wires)
        entangler(weights, anz_wires, rot=rot)
        qml.Barrier(wires)
        swap_test(trash_wires, zero_wires, ancila_wire)
        qml.Barrier(wires)
        return qml.expval(qml.PauliZ(ancila_wire))

    return _qae_inv_decoder

# Stage 2: Stacked Model (Encoder → Decoder in serial)
def qae_stacked_model_shape(n_latent, n_trash, n_layers=1, rot='Rxyz'):
    return entangler_shape(n_layers, n_latent+n_trash, rot=rot)

def qae_stacked_model(wires, dec_weights, n_latent, n_trash, n_layers=1, rot='Rxyz'):
    n_zero = n_trash
    latent_wires = wires[0:n_latent]
    trash_wires = wires[n_latent:n_latent+n_trash]
    zero_wires = wires[n_latent+n_trash:n_latent+n_trash+n_zero]
    data_wires = latent_wires + trash_wires

    def _qae_stacked(weights, inputs):
        # Encoder
        sequence_encoder(data_wires, inputs)
        entangler(weights, data_wires, rot=rot)
        
        # Trash SWAP
        qml.Barrier(wires)
        swap(trash_wires, zero_wires)
        qml.Barrier(wires)
        
        # Decoder (inverted entangler)
        qml.adjoint(entangler)(dec_weights, data_wires, rot=rot)
        
        # Measure all data qubits
        return [qml.expval(qml.PauliZ(m)) for m in data_wires]

    return _qae_stacked

# Stage 3: Full QAE Model (same as Stacked Stage 2, but both weights fixed)
def qae_full_model(wires, enc_weights, dec_weights, n_latent, n_trash, n_layers, rot):
    n_zero = n_trash
    latent_wires = wires[0:n_latent]
    trash_wires = wires[n_latent:n_latent+n_trash]
    zero_wires = wires[n_latent+n_trash:n_latent+n_trash+n_zero]
    data_wires = latent_wires + trash_wires

    def _qae_full(inputs):
        # Encoder
        sequence_encoder(data_wires, inputs)
        entangler(enc_weights, data_wires, rot=rot)
        
        # Trash SWAP
        qml.Barrier(wires)
        swap(trash_wires, zero_wires)
        qml.Barrier(wires)
        
        # Decoder
        qml.adjoint(entangler)(dec_weights, data_wires, rot=rot)
        
        return [qml.expval(qml.PauliZ(m)) for m in data_wires]

    return _qae_full

shape_s1 = qae_inv_decoder_model_shape(n_latent, n_trash, n_layers=1, rot=rot)
shape_s2 = qae_stacked_model_shape(n_latent, n_trash, n_layers=1, rot=rot)
print(f'Stage 1 (Inv Decoder) shape: {shape_s1}')
print(f'Stage 2 (Stacked Encoder) shape: {shape_s2}')
print(f'Architecture: {arch_tag()}')

In [ ]:
# ===========================
# Cell 4 — Cost Functions & Training Utilities
# ===========================

# SWAP test cost (for Stage 1)
def cost_probs_on_ancila(expvals):
    """SWAP test cost function: 1 - P(|1>)"""
    cost = 0
    vals = len(expvals)
    for i in range(vals):
        p1 = (1 - expvals[i]) / 2
        cost = cost + p1
    return cost / vals

def cost_fun_gen_on_probs(model, cost_fun):
    """Generate cost function for SWAP test model"""
    def _cost_fun(params, inputs):
        expvals = pnp.stack([model(params, x) for x in inputs])
        return cost_fun(expvals)
    return _cost_fun

# MSE cost (for Stage 2 - Stacked)
def mse_cost_on_tensors(targets, predictions):
    """Pure MSE cost for expectation values"""
    cost = 0
    vals = 0
    for i in range(len(targets)):
        for w in range(len(targets[i])):
            cost = cost + (targets[i][w] - predictions[i][w]) ** 2
            vals += 1
    return cost / vals

def cost_fun_gen_on_tensors(model, cost_fun):
    """Generate cost function for MSE on tensors"""
    def _cost_fun(params, inputs, targets):
        preds = [model(params, x) for x in inputs]
        return cost_fun(targets, preds)
    return _cost_fun

def get_mini_batches(W_noisy, W_clean, batch_size=10, shuffle=True, seed=0):
    """Generate mini-batches for training"""
    if seed == 0:
        seed = int(time.time()*1000) % 10000
    np.random.seed(seed)
    num_samples = W_clean.shape[0]
    indices = np.arange(num_samples)
    if shuffle:
        np.random.shuffle(indices)
    
    for i in range(0, num_samples, batch_size):
        batch_indices = indices[i:i + batch_size]
        yield W_noisy[batch_indices], W_clean[batch_indices]

def create_sw_tens(X, y, noise=0.0, wind_size=5, wind_step=2, 
                   range_low=0.2, range_high=0.8, seed=0, noise_clip=True):
    """Create sliding window tensors with optional noise"""
    y_noisy = ts_add_noise(y, noise=noise, noise_type='normal', clip=noise_clip,
                          range_low=range_low, range_high=range_high, seed=seed)
    y_ts = ts_wind_make(y_noisy, wind_size, wind_step)
    X_ts = ts_wind_make(X, wind_size, wind_step)
    X_train_ts, y_train_ts, X_test_ts, y_test_ts = ts_wind_split(X_ts, y_ts, split)
    
    X_train_tens = pnp.array(X_train_ts, requires_grad=False)
    y_train_tens = pnp.array(y_train_ts, requires_grad=False)
    X_test_tens = pnp.array(X_test_ts, requires_grad=False)
    y_test_tens = pnp.array(y_test_ts, requires_grad=False)
    return X_train_tens, y_train_tens, X_test_tens, y_test_tens

# Stage 1: Inverted Decoder Training (same as Sidekick)
def train_inv_decoder(model, X, cost_fun, optimizer, n_epochs, init_weights,
                      log_interv=1, prompt_fract=0.1, start_time=0, seed=0,
                      wind_size=8, wind_step=4, enc_lim=(0, 1)):
    """Train inverted decoder without noise"""
    scale_low, scale_high = enc_lim
    
    if seed == 0:
        seed = int(time.time()*1000) % 10000
    np.random.seed(seed)
    
    hist_cost = []
    hist_params = []
    params = init_weights.copy()
    
    _, y_pure_tens, _, _ = create_sw_tens(X, X, noise=0, seed=seed,
        wind_size=wind_size, wind_step=wind_step, range_low=scale_low, 
        range_high=scale_high, noise_clip=noise_clip)
    
    if start_time == 0:
        start_time = time.time()
    
    for iter in range(n_epochs):
        params, cost = optimizer.step_and_cost(lambda p: cost_fun(p, y_pure_tens), params)
        elapsed_time = time.time() - start_time
        
        if iter % log_interv == 0:
            hist_cost.append(cost)
            hist_params.append(params)
        
        if (prompt_fract > 0) and (iter % int(prompt_fract*n_epochs) == 0):
            print(f'Iter: {iter:03d} ({int(elapsed_time):04d} sec) cost={cost:0.6f}')
    
    min_cost = np.min(hist_cost)
    min_iter = np.argmin(hist_cost)
    elapsed = int(time.time() - start_time)
    print(f'\nStage 1 completed: epochs={n_epochs}, min cost={min_cost:.6f} @ {min_iter}, ' +
          f'time={elapsed} sec\n')
    
    return hist_cost, hist_params, init_weights, (min_iter, min_cost, elapsed)

# Stage 2: Stacked Training (MSE cost on expectation values)
def train_stacked_with_noise(model, W, cost_fun, optimizer, n_epochs, init_weights,
                              log_interv=1, prompt_fract=0.1, start_time=0, seed=0,
                              wind_size=8, wind_step=4, noise=0,
                              enc_lim=(0, 1), dec_lim=(0, 1)):
    """Train stacked encoder with noisy mini-batches using MSE cost"""
    enc_low, enc_high = enc_lim
    dec_low, dec_high = dec_lim
    
    if seed == 0:
        seed = int(time.time()*1000) % 10000
    np.random.seed(seed)
    
    hist_cost = []
    hist_params = []
    params = init_weights.copy()
    
    # Prepare pure windows as TARGETS (scaled to dec range)
    _, y_pure_tens, _, _ = create_sw_tens(W, W, noise=0, seed=seed,
        wind_size=wind_size, wind_step=wind_step, range_low=enc_low, 
        range_high=enc_high, noise_clip=noise_clip)
    
    # Scale targets to expectation value range [-1, 1]
    y_pure_tens = x2y(y_pure_tens, xlim=(enc_low, enc_high), ylim=(dec_low, dec_high))
    
    if start_time == 0:
        start_time = time.time()
    
    for iter in range(n_epochs):
        # Generate noisy windows as INPUTS
        _, X_noisy_tens, _, _ = create_sw_tens(W, W, noise=noise, seed=seed+iter,
            wind_size=wind_size, wind_step=wind_step, range_low=enc_low, 
            range_high=enc_high, noise_clip=noise_clip)
        
        # Decay learning rate
        if (iter+1) % lr_stacked_decay_steps == 0:
            optimizer.stepsize *= lr_stacked_decay_rate
        
        # Train in mini-batches
        batches = get_mini_batches(X_noisy_tens, y_pure_tens, batch_size=batch_size,
                                   shuffle=True, seed=seed+iter)
        acc_batch_cost = 0
        for batch_no, (noisy_batch, pure_batch) in enumerate(batches):
            params, batch_cost = optimizer.step_and_cost(
                lambda p: cost_fun(p, noisy_batch, pure_batch), params)
            acc_batch_cost += batch_cost
        
        cost = acc_batch_cost / (batch_no + 1)
        
        if iter % log_interv == 0:
            hist_cost.append(cost)
            hist_params.append(params)
        
        if (prompt_fract > 0) and (iter % int(prompt_fract*n_epochs) == 0):
            elapsed = int(time.time() - start_time)
            print(f'Iter: {iter:03d} ({elapsed:04d} sec) cost={cost:0.6f}, ' +
                  f'min={np.min(hist_cost):0.6f} (LR={optimizer.stepsize:0.4f})')
    
    min_cost = np.min(hist_cost)
    min_iter = np.argmin(hist_cost)
    elapsed = int(time.time() - start_time)
    print(f'\nStage 2 completed: epochs={n_epochs}, min cost={min_cost:.6f} @ {min_iter}, ' +
          f'time={elapsed} sec\n')
    
    return hist_cost, hist_params, init_weights, (min_iter, min_cost, elapsed)

# Evaluation helpers
def _flatten_avg(windows):
    """Flatten overlapping windows by averaging"""
    return ts_wind_flatten_avg(np.asarray(windows), wind_step)

def _mse(a, b):
    """Calculate MSE between two sequences"""
    a = np.asarray(a)
    b = np.asarray(b)
    return float(np.mean((a - b) ** 2))

def windows_from_seed(y_vec, sigma, seed, clip=True):
    """Generate clean and noisy windows with fixed seed"""
    y_noisy = ts_add_noise(y_vec, noise=sigma, noise_type='normal', clip=clip,
                          range_low=scale_low, range_high=scale_high, seed=seed)
    win_clean = ts_wind_make(y_vec, wind_size, wind_step)
    win_noisy = ts_wind_make(y_noisy, wind_size, wind_step)
    X_ts = np.array([i*wind_step for i in range(len(win_clean))])
    _, tr_clean, _, te_clean = ts_wind_split(X_ts, win_clean, split)
    _, tr_noisy, _, te_noisy = ts_wind_split(X_ts, win_noisy, split)
    return tr_clean, tr_noisy, te_clean, te_noisy

def run_qnode_series(model, windows):
    """Run model on series of windows (no params for full QAE)"""
    outs = []
    for w in windows:
        z = model(w)
        outs.append(np.asarray(z))
    return np.stack(outs)

def render_circuit_stage1(L, weights_vec, save_dir, inst, seed=None):
    """Render Stage 1 (inverted decoder) circuit"""
    wires = list(range(n_latent + 2*n_trash + 1))
    qae = qae_inv_decoder_model(wires, n_latent, n_trash, n_layers=L, rot=rot)
    
    if seed is not None:
        dev = qml.device(sim, wires=len(wires), shots=shots, seed=seed)
    else:
        dev = qml.device(sim, wires=len(wires), shots=shots)
    
    qnode = qml.QNode(qae, dev)
    
    x_dummy = np.zeros(wind_size, dtype=float)
    qml.drawer.use_style("pennylane")
    fig_func = qml.draw_mpl(qnode, decimals=2, level="device")
    fig, ax = fig_func(weights_vec, x_dummy)
    ax.set_title(f'Stacked Stage 1: Inv Decoder (L={L}, inst={inst})')
    
    save_dir = Path(save_dir)
    save_dir.mkdir(parents=True, exist_ok=True)
    out_eps = save_dir / f"circuit_stage1_{arch_tag()}_L{L}_inst{inst:02d}.eps"
    fig.savefig(out_eps, dpi=220, bbox_inches="tight", pad_inches=0.1, format="eps")
    plt.close(fig)
    return out_eps

def render_circuit_stage2(L, dec_weights, enc_weights, save_dir, inst, seed=None):  # ✅ ADDED seed param
    """Render Stage 2 (stacked) circuit"""
    wires = list(range(n_latent + 2*n_trash))
    qae = qae_stacked_model(wires, dec_weights, n_latent, n_trash, n_layers=L, rot=rot)
    
    if seed is not None:
        dev = qml.device(sim, wires=len(wires), shots=shots, seed=seed)
    else:
        dev = qml.device(sim, wires=len(wires), shots=shots)
    
    qnode = qml.QNode(qae, dev)
    
    x_dummy = np.zeros(wind_size, dtype=float)
    qml.drawer.use_style("pennylane")
    fig_func = qml.draw_mpl(qnode, decimals=2, level="device")
    fig, ax = fig_func(enc_weights, x_dummy)
    ax.set_title(f'Stacked Stage 2: Full Stacked (L={L}, inst={inst})')
    
    save_dir = Path(save_dir)
    save_dir.mkdir(parents=True, exist_ok=True)
    out_eps = save_dir / f"circuit_stage2_{arch_tag()}_L{L}_inst{inst:02d}.eps"
    fig.savefig(out_eps, dpi=220, bbox_inches="tight", pad_inches=0.1, format="eps")
    plt.close(fig)
    return out_eps

def render_circuit_full(L, enc_weights, dec_weights, save_dir, inst, seed=None):  # ✅ ADDED seed param
    """Render full QAE circuit"""
    wires = list(range(n_wires_full))
    qae = qae_full_model(wires, enc_weights, dec_weights, n_latent, n_trash, L, rot)
    
    if seed is not None:
        dev = qml.device(sim, wires=n_wires_full, shots=shots, seed=seed)
    else:
        dev = qml.device(sim, wires=n_wires_full, shots=shots)
    
    qnode = qml.QNode(qae, dev)
    
    x_dummy = np.zeros(wind_size, dtype=float)
    qml.drawer.use_style("pennylane")
    fig_func = qml.draw_mpl(qnode, decimals=2, level="device")
    fig, ax = fig_func(x_dummy)
    ax.set_title(f'Stacked Full QAE (L={L}, inst={inst})')
    
    save_dir = Path(save_dir)
    save_dir.mkdir(parents=True, exist_ok=True)
    out_eps = save_dir / f"circuit_full_{arch_tag()}_L{L}_inst{inst:02d}.eps"
    fig.savefig(out_eps, dpi=220, bbox_inches="tight", pad_inches=0.1, format="eps")
    plt.close(fig)
    return out_eps

In [ ]:
# ===========================
# Cell 5 — Multi-Instance Training
# ===========================
import pandas as pd

def ensure_dir(p):
    """Create directory if it doesn't exist"""
    p = Path(p)
    p.mkdir(parents=True, exist_ok=True)
    return p

# Setup directory structure
ROOT = ensure_dir(BUNDLE_ROOT)
DATA_ROOT = ensure_dir(ROOT / DATA_ID)
ARCH_ROOT = ensure_dir(DATA_ROOT / arch_tag())
CSV_RUNS = ARCH_ROOT / "metrics_by_run.csv"
CSV_SUMMARY = ARCH_ROOT / "summary_by_layer.csv"

RUNS_HEADER = [
    "dataset", "arch", "instance_id", "layer",
    "window_size", "step", "sigma_eval",
    "stage1_min_cost", "stage1_time_sec",
    "stage2_min_cost", "stage2_time_sec",
    "mse_train_noise", "mse_train_recovered", "delta_train_pct",
    "mse_test_noise", "mse_test_recovered", "delta_test_pct",
    "mse_train_recovered_pure", "mse_test_recovered_pure",
    "timestamp", "bundle_json"
]

if not CSV_RUNS.exists():
    with open(CSV_RUNS, "w", newline="", encoding="utf-8") as f:
        csv.writer(f).writerow(RUNS_HEADER)

ALL_RUNS = []

# Encode data once
y_enc = x2y(y, xlim=(scale_low, scale_high), ylim=(y_enc_low, y_enc_high))

for L in LAYER_OPTIONS:
    print(f"\n{'='*60}")
    print(f"TRAINING LAYER {L}")
    print(f"{'='*60}")
    
    # Get shapes
    shape_s1 = qae_inv_decoder_model_shape(n_latent, n_trash, n_layers=L, rot=rot)
    shape_s2 = qae_stacked_model_shape(n_latent, n_trash, n_layers=L, rot=rot)
    
    layer_dir = ensure_dir(ARCH_ROOT / f"L{L}")
    
    for inst in INSTANCES_PER_LAYER:
        print(f"\n{'─'*60}")
        print(f"Layer {L} | Instance {inst}")
        print(f"{'─'*60}")
        
        
        seed_info = set_global_seed(inst, layer=L)
        device_seed = seed_info['device_seed']
        print(f"Seeds: global={seed_info['global_seed']}, device={device_seed}")
        
        # ============================================================
        # STAGE 1: Train Inverted Decoder
        # ============================================================
        print(f"\n▶ STAGE 1: Training Inverted Decoder")
        
        wires_s1 = list(range(n_latent + 2*n_trash + 1))
        dev_s1 = qml.device(sim, wires=len(wires_s1), shots=shots, seed=device_seed)
        qae_inv_dec = qae_inv_decoder_model(wires_s1, n_latent, n_trash, n_layers=L, rot=rot)
        qae_inv_dec_qc = qml.QNode(qae_inv_dec, dev_s1, interface=interface, diff_method=diff_method)
        
        init_weights_s1 = pnp.array(
            np.random.uniform(high=2*np.pi, size=shape_s1) * inv_decoder_weight_scaler,
            requires_grad=True)
        
        opt_s1 = qml.AdamOptimizer(stepsize=lr_inv_decoder_initial)
        cost_fun_s1 = cost_fun_gen_on_probs(qae_inv_dec_qc, cost_probs_on_ancila)
        
        hist_cost_s1, hist_params_s1, _, stats_s1 = train_inv_decoder(
            qae_inv_dec_qc, y_enc, cost_fun_s1, opt_s1, n_inv_decoder_epochs,
            init_weights_s1, log_interv=log_interv, prompt_fract=0.1,
            seed=TRAIN_SEED_BASE, wind_size=wind_size, wind_step=wind_step,
            enc_lim=(y_enc_low, y_enc_high))
        
        min_iter_s1, min_cost_s1, elapsed_s1 = stats_s1
        opt_weights_s1 = hist_params_s1[min_iter_s1]
        
        # ============================================================
        # STAGE 2: Train Stacked Encoder
        # ============================================================
        print(f"\n▶ STAGE 2: Training Stacked Encoder")
        
        wires_s2 = list(range(n_latent + 2*n_trash))
        dev_s2 = qml.device(sim, wires=len(wires_s2), shots=shots, seed=device_seed)
        qae_stacked = qae_stacked_model(wires_s2, opt_weights_s1, n_latent, n_trash, n_layers=L, rot=rot)
        qae_stacked_qc = qml.QNode(qae_stacked, dev_s2, interface=interface, diff_method=diff_method)
        
        # Initialize encoder with decoder weights (mirror strategy)
        init_weights_s2 = opt_weights_s1.copy()
        
        opt_s2 = qml.AdamOptimizer(stepsize=lr_stacked_initial, beta1=0.99)
        cost_fun_s2 = cost_fun_gen_on_tensors(qae_stacked_qc, mse_cost_on_tensors)
        
        hist_cost_s2, hist_params_s2, _, stats_s2 = train_stacked_with_noise(
            qae_stacked_qc, y_enc, cost_fun_s2, opt_s2, n_stacked_epochs,
            init_weights_s2, log_interv=log_interv, prompt_fract=0.1,
            seed=TRAIN_SEED_BASE, wind_size=wind_size, wind_step=wind_step,
            noise=noise, enc_lim=(y_enc_low, y_enc_high), dec_lim=(y_dec_low, y_dec_high))
        
        min_iter_s2, min_cost_s2, elapsed_s2 = stats_s2
        opt_weights_s2 = hist_params_s2[min_iter_s2]
        
        # ============================================================
        # STAGE 3: Evaluate Full QAE
        # ============================================================
        print(f"\n▶ STAGE 3: Evaluating Full QAE")
        
        wires_full = list(range(n_wires_full))
        dev_full = qml.device(sim, wires=n_wires_full, shots=shots, seed=device_seed)
        qae_full = qae_full_model(wires_full, opt_weights_s2, opt_weights_s1, n_latent, n_trash, L, rot)
        qae_full_qc = qml.QNode(qae_full, dev_full, interface=interface, diff_method=diff_method)
        
        # Generate test data with fixed seed
        tr_c, tr_n, te_c, te_n = windows_from_seed(y, noise, TEST_SEED_FIXED, clip=True)
        tr_n_enc = x2y(tr_n, xlim=(scale_low, scale_high), ylim=(y_enc_low, y_enc_high))
        te_n_enc = x2y(te_n, xlim=(scale_low, scale_high), ylim=(y_enc_low, y_enc_high))
        
        tr_c_enc = x2y(tr_c, xlim=(scale_low, scale_high), ylim=(y_enc_low, y_enc_high))
        te_c_enc = x2y(te_c, xlim=(scale_low, scale_high), ylim=(y_enc_low, y_enc_high))
        
        # Run inference
        tr_hat = run_qnode_series(qae_full_qc, tr_n_enc)
        te_hat = run_qnode_series(qae_full_qc, te_n_enc)
        tr_hat_pure = run_qnode_series(qae_full_qc, tr_c_enc)
        te_hat_pure = run_qnode_series(qae_full_qc, te_c_enc)
        
        # Flatten and rescale
        tr_pure = _flatten_avg(tr_c)
        te_pure = _flatten_avg(te_c)
        tr_noi = _flatten_avg(tr_n)
        te_noi = _flatten_avg(te_n)
        tr_rec = x2y(_flatten_avg(tr_hat), xlim=(y_dec_low, y_dec_high), ylim=(scale_low, scale_high))
        te_rec = x2y(_flatten_avg(te_hat), xlim=(y_dec_low, y_dec_high), ylim=(scale_low, scale_high))
        tr_rec_pure = x2y(_flatten_avg(tr_hat_pure), xlim=(y_dec_low, y_dec_high), ylim=(scale_low, scale_high))
        te_rec_pure = x2y(_flatten_avg(te_hat_pure), xlim=(y_dec_low, y_dec_high), ylim=(scale_low, scale_high))
        
        # Compute metrics
        mse_tr_noise = _mse(tr_pure, tr_noi)
        mse_te_noise = _mse(te_pure, te_noi)
        mse_tr_rec = _mse(tr_pure, tr_rec)
        mse_te_rec = _mse(te_pure, te_rec)
        mse_tr_rec_pure = _mse(tr_pure, tr_rec_pure)
        mse_te_rec_pure = _mse(te_pure, te_rec_pure)
        
        delta_tr = 100.0 * (1.0 - mse_tr_rec / max(mse_tr_noise, 1e-12))
        delta_te = 100.0 * (1.0 - mse_te_rec / max(mse_te_noise, 1e-12))
        
        print(f"Test MSE: noise={mse_te_noise:.6f}, recovered={mse_te_rec:.6f}, Δ={delta_te:+.2f}%")
        
        # Save artifacts
        inst_tag = f"inst{inst:02d}_L{L}"
        
        np.save(layer_dir / f"weights_s1_{inst_tag}.npy", np.array(opt_weights_s1))
        np.save(layer_dir / f"weights_s2_{inst_tag}.npy", np.array(opt_weights_s2))
        
        circuit_s1_path = render_circuit_stage1(L, np.array(opt_weights_s1), ARCH_ROOT, inst, seed=device_seed)
        circuit_s2_path = render_circuit_stage2(L, np.array(opt_weights_s1), np.array(opt_weights_s2), ARCH_ROOT, inst, seed=device_seed)
        circuit_full_path = render_circuit_full(L, np.array(opt_weights_s2), np.array(opt_weights_s1), ARCH_ROOT, inst, seed=device_seed)
        
        # Save training costs
        cost_s1_plot = layer_dir / f"training_cost_s1_{inst_tag}.eps"
        meas_plot(hist_cost_s1, meas='cost', task='min',
                  title_pref=f'L{L} Inst {inst} Stage 1 Training',
                  ylim=(0, max(0.15, max(hist_cost_s1)*1.1)), rcParams=(10, 4),
                  log_interv=log_interv, backplot=True, back_color='lightgray',
                  smooth_weight=0.5, save_plot=cost_s1_plot)
        
        cost_s2_plot = layer_dir / f"training_cost_s2_{inst_tag}.eps"
        meas_plot(hist_cost_s2, meas='cost', task='min',
                  title_pref=f'L{L} Inst {inst} Stage 2 Training',
                  ylim=(0, max(0.15, max(hist_cost_s2)*1.1)), rcParams=(10, 4),
                  log_interv=log_interv, backplot=True, back_color='lightgray',
                  smooth_weight=0.9, save_plot=cost_s2_plot)
        
        # Save cost histories
        with open(layer_dir / f"cost_s1_{inst_tag}.json", "w") as f:
            json.dump([float(c) for c in hist_cost_s1], f, indent=2)
        with open(layer_dir / f"cost_s2_{inst_tag}.json", "w") as f:
            json.dump([float(c) for c in hist_cost_s2], f, indent=2)
        
        # Create coordinate arrays
        n_train = len(tr_pure)
        n_test = len(te_pure)
        X_coords_train = np.arange(0, n_train * wind_step, wind_step)[:n_train]
        X_coords_test = np.arange(n_train * wind_step,
                                  n_train * wind_step + n_test * wind_step,
                                  wind_step)[:n_test]
        
        # Plot reconstruction from pure data
        recon_pure_path = layer_dir / f"reconstruction_pure_{inst_tag}.eps"
        multi_plot_flat_ts(
            X_list=[X_coords_train, X_coords_test, X_coords_train, X_coords_test, X_coords_train, X_coords_test],
            y_list=[tr_noi, te_noi, tr_pure, te_pure, tr_rec_pure, te_rec_pure],
            labels=[f"Noisy train (MSE {mse_tr_noise:0.4f})",
                    f"Noisy test  (MSE {mse_te_noise:0.4f})",
                    "True train", "True test",
                    f"Recovered train (MSE {mse_tr_rec_pure:0.4f})",
                    f"Recovered test  (MSE {mse_te_rec_pure:0.4f})"],
            vert_lines=['', 'dotted', '', '', '', ''], 
            vert_line_color='gray',
            colors=['lightblue', 'pink', 'cornflowerblue', 'salmon', 'blue', 'red'],
            marker_colors=['white', 'white', 'lightblue', 'mistyrose', 'blue', 'red'],
            lines=['', '', 'solid', 'solid', 'dashed', 'dashed'],
            markers=['.', '.', '', '', '', ''],
            rcParams=(12, 6), dpi=200,
            xlabel=f'Time (windows: size={wind_size}, step={wind_step})', 
            ylabel='y Value',
            legend_cols=3, 
            title=f"Pure data vs recovered from pure data (L={L}, inst={inst})",
            save_plot=recon_pure_path)
        
        # Plot reconstruction from noisy data
        recon_noisy_path = layer_dir / f"reconstruction_noisy_{inst_tag}.eps"
        multi_plot_flat_ts(
            X_list=[X_coords_train, X_coords_test, X_coords_train, X_coords_test, X_coords_train, X_coords_test],
            y_list=[tr_noi, te_noi, tr_pure, te_pure, tr_rec, te_rec],
            labels=[f"Noisy train (MSE {mse_tr_noise:0.4f})",
                    f"Noisy test  (MSE {mse_te_noise:0.4f})",
                    "True train", "True test",
                    f"Recovered train (MSE {mse_tr_rec:0.4f})",
                    f"Recovered test  (MSE {mse_te_rec:0.4f})"],
            vert_lines=['', 'dotted', '', '', '', ''], 
            vert_line_color='gray',
            colors=['lightblue', 'pink', 'cornflowerblue', 'salmon', 'blue', 'red'],
            marker_colors=['white', 'white', 'lightblue', 'mistyrose', 'blue', 'red'],
            lines=['', '', 'solid', 'solid', 'dashed', 'dashed'],
            markers=['.', '.', '', '', '', ''],
            rcParams=(12, 6), dpi=200,
            xlabel=f'Time (windows: size={wind_size}, step={wind_step})', 
            ylabel='y Value',
            legend_cols=3, 
            title=f"Pure data vs recovered from noise (L={L}, inst={inst})",
            save_plot=recon_noisy_path)
        
        # Create bundle
        bundle = {
            "schema": {"name": "stacked_qae_bundle", "version": "1.0"},
            "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
            "dataset": {
                "id": DATA_ID,
                "scale_low": float(scale_low),
                "scale_high": float(scale_high),
                "window_size": int(wind_size),
                "window_step": int(wind_step),
                "split": float(split),
            },
            "run": {
                "instance_id": int(inst),
                "layer": int(L),
                "sigma_eval": float(noise),
                "test_seed_fixed": int(TEST_SEED_FIXED),
                "device_seed": int(device_seed),
                "stage1_time_sec": int(elapsed_s1),
                "stage2_time_sec": int(elapsed_s2),
            },
            "environment": {
                "device": sim,
                "diff_method": diff_method,
                "interface": interface,
            },
            "architecture": {
                "type": "stacked",
                "n_qubits": int(wind_size),
                "n_latent": int(n_latent),
                "n_trash": int(n_trash),
                "rot": str(rot),
            },
            "parameters": {
                "weights_stage1": np.array(opt_weights_s1).tolist(),
                "weights_stage2": np.array(opt_weights_s2).tolist(),
            },
            "metrics": {
                "stage1_min_cost": float(min_cost_s1),
                "stage2_min_cost": float(min_cost_s2),
                "mse_train_noise": float(mse_tr_noise),
                "mse_train_recovered": float(mse_tr_rec),
                "delta_train_pct": float(delta_tr),
                "mse_test_noise": float(mse_te_noise),
                "mse_test_recovered": float(mse_te_rec),
                "delta_test_pct": float(delta_te),
                "mse_train_recovered_pure": float(mse_tr_rec_pure),
                "mse_test_recovered_pure": float(mse_te_rec_pure),
            },
            "artifacts": {
                "weights_s1_file": f"weights_s1_{inst_tag}.npy",
                "weights_s2_file": f"weights_s2_{inst_tag}.npy",
                "circuit_stage1": str(circuit_s1_path.name),
                "circuit_stage2": str(circuit_s2_path.name),
                "circuit_full": str(circuit_full_path.name),
                "training_cost_s1_plot": str(cost_s1_plot.name),
                "training_cost_s2_plot": str(cost_s2_plot.name),
                "reconstruction_pure_plot": str(recon_pure_path.name),
                "reconstruction_noisy_plot": str(recon_noisy_path.name),
            }
        }
        
        bundle_path = layer_dir / f"bundle_{inst_tag}.json"
        with open(bundle_path, "w", encoding="utf-8") as f:
            json.dump(bundle, f, indent=2)
        
        # Save to CSV
        row = [
            DATA_ID, arch_tag(), int(inst), int(L),
            int(wind_size), int(wind_step), float(noise),
            float(min_cost_s1), int(elapsed_s1),
            float(min_cost_s2), int(elapsed_s2),
            mse_tr_noise, mse_tr_rec, delta_tr,
            mse_te_noise, mse_te_rec, delta_te,
            mse_tr_rec_pure, mse_te_rec_pure,
            bundle["timestamp"], str(bundle_path)
        ]
        
        if CSV_RUNS.exists():
            df_old = pd.read_csv(CSV_RUNS)
            df_new = pd.concat([df_old, pd.DataFrame([row], columns=RUNS_HEADER)],
                              ignore_index=True)
            df_new.to_csv(CSV_RUNS, index=False)
        else:
            pd.DataFrame([row], columns=RUNS_HEADER).to_csv(CSV_RUNS, index=False)
        
        ALL_RUNS.append({
            "instance_id": inst,
            "layer": L,
            "weights_s1": np.array(opt_weights_s1),
            "weights_s2": np.array(opt_weights_s2)
        })

print(f"\n{'='*60}")
print(f"TRAINING COMPLETE")
print(f"Metrics saved: {CSV_RUNS}")
print(f"Artifacts saved: {ARCH_ROOT}")
print(f"{'='*60}")

# Generate summary
if CSV_RUNS.exists():
    df = pd.read_csv(CSV_RUNS)
    summary = (df.groupby("layer", as_index=True)
               .agg(runs=("instance_id", "count"),
                    avg_stage1_time=("stage1_time_sec", "mean"),
                    avg_stage2_time=("stage2_time_sec", "mean"),
                    mse_test_noise_mean=("mse_test_noise", "mean"),
                    mse_test_recovered_mean=("mse_test_recovered", "mean"),
                    delta_test_pct_mean=("delta_test_pct", "mean")))
    summary.to_csv(CSV_SUMMARY)
    print(f"\nSummary saved: {CSV_SUMMARY}")
    print(summary)